Задача: 

С помощью методов машинного обучения выявить сомнительные операции клиентов на основе данных по транзакциям и вердиктов, проставленных вручную экспертами фрод-мониторинга.


In [ ]:
!pip install dask_ml pandas

In [23]:
import pandas as pd

In [24]:
df_data_trx = pd.read_csv("hits0712.csv", index_col=0)

In [25]:
df_data_trx.shape

(9016, 6)

In [26]:
df_data_trx.head(5)

,event_id,user_id,rec_user_id,date_time,resolution,sub_channel
5,0fcea7fec4ff479cac8cf37b4555c817,75301207,49913000,2018-12-07 07:56:49,G,ATMAPI
6,c29a4e64d27b435b9b55aa3e62ce54d4,1200695,12797310,2018-12-07 07:14:41,G,MOBILEAPI
8,3107b2b1afcb490ab0a31135eb4b386c,45657780,23814017,2018-12-07 14:02:30,G,MOBILEAPI
10,d9bc95d991144d53b950d8084fa846bf,21683486,9865131,2018-12-07 10:32:41,G,WEBAPI
22,9fc959da4cd144ea8bf5bae2dc041a5e,4531894,VSP15472255,2018-12-07 18:22:52,G,MOBILEAPI


In [27]:
df_data_trx.resolution.value_counts(dropna=False)

G    8943
F      63
S      10
Name: resolution, dtype: int64

In [28]:
target = df_data_trx[["rec_user_id", "resolution"]].drop_duplicates()
target.head()

,rec_user_id,resolution
5,49913000,G
6,12797310,G
8,23814017,G
10,9865131,G
22,VSP15472255,G


In [29]:
resolution_map = {"G": 0, "F": 1, "S": 1}
target["class"] = target.resolution.map(lambda x: resolution_map[x])

In [30]:
target["class"].value_counts()

0    7210
1      52
Name: class, dtype: int64

In [31]:
df_data_hist = pd.read_csv("hist_trx.csv")

In [32]:
df_data_hist.head()

,Unnamed: 0,event_id,date_time,user_id,sub_channel,event_type,sub_type,atm_mcc,mcc_group,atm_merchant_name,amount
0,0,22c9f1ac686a43e18cdb798489193238,2018-12-06 09:33:21,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5921.0,R,YUG 426,280.0
1,1,62dac13fac68416d9bc340c51ddcb977,2018-12-06 09:36:08,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5331.0,R,MAGAZIN RODINA,376.9
2,2,9b666b8c9d4d4faea78e2b28a5468794,2018-12-06 12:21:02,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5921.0,R,YUG 426,143.0
3,3,d4d805fc3d5f4c91aa9b0389333b780c,2018-12-06 07:46:58,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5331.0,R,MAGAZIN RODINA,162.0
4,4,1d162351117a4418949dcf94111b9964,2018-12-06 12:21:27,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5331.0,R,MAGAZIN RODINA,56.0


In [33]:
user_mcc_group = df_data_hist[["Unnamed: 0", "user_id", "mcc_group"]]

In [34]:
user_mcc_group

,Unnamed: 0,user_id,mcc_group
0,0,102050167,R
1,1,102050167,R
2,2,102050167,R
3,3,102050167,R
4,4,102050167,R
...,...,...,...
484136,29729,CRM21933741,R
484137,29730,MBK11292037,F
484138,29731,MBK11292037,C
484139,29732,MBK5364257,R


In [35]:
user_mcc_group.mcc_group.value_counts(dropna=False)

R      164727
C      127550
U      100337
Z       59449
F       21065
X        4882
T        2429
A        1452
NaN       952
H         665
J         446
Q         124
O          63
Name: mcc_group, dtype: int64

In [36]:
oec_data = pd.pivot_table(user_mcc_group, index="user_id", columns=["mcc_group"], values="Unnamed: 0", aggfunc="count", dropna=True, fill_value=0)

In [37]:
target_all = pd.merge(oec_data, target[["rec_user_id", "class"]], right_on="rec_user_id", left_on="user_id")

In [38]:
target_all.head()

,A,C,F,H,J,O,Q,R,T,U,X,Z,rec_user_id,class
0,0,5,0,0,0,0,0,14,0,8,1,40,100017681,0
1,0,23,0,0,0,0,0,28,0,19,0,0,100023450,0
2,0,1,0,0,0,0,0,5,0,0,0,0,100031471,0
3,0,24,5,0,0,0,0,14,0,36,1,7,10008487,0
4,0,4,0,0,0,0,0,7,0,3,0,0,10009395,0


In [39]:
target_all[target_all["class"]!=1]

,A,C,F,H,J,O,Q,R,T,U,X,Z,rec_user_id,class
0,0,5,0,0,0,0,0,14,0,8,1,40,100017681,0
1,0,23,0,0,0,0,0,28,0,19,0,0,100023450,0
2,0,1,0,0,0,0,0,5,0,0,0,0,100031471,0
3,0,24,5,0,0,0,0,14,0,36,1,7,10008487,0
4,0,4,0,0,0,0,0,7,0,3,0,0,10009395,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6789,0,2,3,0,0,0,0,9,1,3,0,5,VSP5217599,0
6790,0,1,1,0,0,0,0,6,0,0,0,1,VSP651163,0
6791,0,0,1,0,0,0,0,8,0,0,1,1,VSP840251,0
6792,0,0,0,0,0,0,0,1,0,1,0,0,VSP847400,0


In [40]:
target_all["class"].value_counts()

0    6747
1      47
Name: class, dtype: int64

In [41]:
Xdata = target_all.drop(["class", "rec_user_id"], axis=1)
ydata = target_all["class"]

In [ ]:
!pip install scikit-learn

In [43]:
from sklearn.model_selection import train_test_split

In [44]:
Xtrain, Xtest, ytrain, ytest = train_test_split(Xdata, ydata, test_size=0.4, random_state=55)

In [45]:
Xtest.shape

(2718, 12)

In [46]:
ytrain.shape

(4076,)

In [47]:
from sklearn.ensemble import RandomForestClassifier

In [48]:
clf = RandomForestClassifier(class_weight={0: 1, 1: 20})
clf.fit(Xtrain, ytrain)

RandomForestClassifier(class_weight={0: 1, 1: 20})

In [49]:
ypred = pd.Series(clf.predict(Xtest), index=Xtest.index)

In [50]:
ytest.value_counts()

0    2702
1      16
Name: class, dtype: int64

In [51]:
ypred.value_counts()

0    2716
1       2
dtype: int64

In [52]:
from sklearn.metrics import confusion_matrix, classification_report

In [53]:
confusion_matrix(ytest, ypred)

array([[2701,    1],
       [  15,    1]])

In [54]:
target_names = ["Genuine", "Fraud"]
print(classification_report(ytest, ypred, target_names=target_names))

              precision    recall  f1-score   support

     Genuine       0.99      1.00      1.00      2702
       Fraud       0.50      0.06      0.11        16

    accuracy                           0.99      2718
   macro avg       0.75      0.53      0.55      2718
weighted avg       0.99      0.99      0.99      2718

